In [3]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import gzip
import re
import nltk
import numpy as np
#nltk.download()
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
pd.options.mode.chained_assignment = None  # default='warn'

In [4]:
#Taken from Julian McAuley: http://jmcauley.ucsd.edu/data/amazon/
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('health_and_personal.json.gz')

In [5]:
print df.shape
df.head()

(346355, 9)


,reviewerID,asin,reviewerName,helpful,unixReviewTime,reviewText,overall,reviewTime,summary
0,ALC5GH8CAMAI7,159985130X,AnnN,"[1, 1]",1294185600,This is a great little gadget to have around. ...,5,"01 5, 2011",Handy little gadget
1,AHKSURW85PJUE,159985130X,"AZ buyer ""AZ buyer""","[1, 1]",1329523200,I would recommend this for a travel magnifier ...,4,"02 18, 2012",Small & may need to encourage battery
2,A38RMU1Y5TDP9,159985130X,"Bob Tobias ""Robert Tobias""","[75, 77]",1275955200,What I liked was the quality of the lens and t...,4,"06 8, 2010",Very good but not great
3,A1XZUG7DFXXOS4,159985130X,Cat lover,"[56, 60]",1202428800,Love the Great point light pocket magnifier! ...,4,"02 8, 2008",great addition to your purse
4,A1MS3M7M7AM13X,159985130X,Cricketoes,"[1, 1]",1313452800,This is very nice. You pull out on the magnifi...,5,"08 16, 2011",Very nice and convenient.


In [6]:
#We need to have some sort of label in order to figure out if a review is helpful or not helpful, so we'll need to define an
#arbitrary ratio + 
a = df['helpful'][:]
helpful = []
notHelpful = []
for i in a:
    helpful.append(i[0])
    notHelpful.append(i[1])
df['helpUp'] = helpful
df['helpDown'] = notHelpful
df['hRatio'] = df.helpUp/df.helpDown
df.head()

,reviewerID,asin,reviewerName,helpful,unixReviewTime,reviewText,overall,reviewTime,summary,helpUp,helpDown,hRatio
0,ALC5GH8CAMAI7,159985130X,AnnN,"[1, 1]",1294185600,This is a great little gadget to have around. ...,5,"01 5, 2011",Handy little gadget,1,1,1.000000
1,AHKSURW85PJUE,159985130X,"AZ buyer ""AZ buyer""","[1, 1]",1329523200,I would recommend this for a travel magnifier ...,4,"02 18, 2012",Small & may need to encourage battery,1,1,1.000000
2,A38RMU1Y5TDP9,159985130X,"Bob Tobias ""Robert Tobias""","[75, 77]",1275955200,What I liked was the quality of the lens and t...,4,"06 8, 2010",Very good but not great,75,77,0.974026
3,A1XZUG7DFXXOS4,159985130X,Cat lover,"[56, 60]",1202428800,Love the Great point light pocket magnifier! ...,4,"02 8, 2008",great addition to your purse,56,60,0.933333
4,A1MS3M7M7AM13X,159985130X,Cricketoes,"[1, 1]",1313452800,This is very nice. You pull out on the magnifi...,5,"08 16, 2011",Very nice and convenient.,1,1,1.000000


In [7]:
#Remove all 0 - 2 reviews
df2 = df.loc[df.helpUp > 2]
df2 = df2.reset_index(drop=True)
#0-25% = useless, 25-50% is unhelpful, 50-75% is moderate,  75-100% is helpful
senti = pd.Series(np.random.randn(len(df2.asin)))
df2 = df2.assign(sentiment = senti.values)
df2.loc[:,'sentiment'][df2['hRatio']<0.25] = 'useless'
df2.loc[:,'sentiment'][(df2['hRatio']>0.25) & (df2['hRatio']<0.50)] = 'unhelpful'
df2.loc[:,'sentiment'][(df2['hRatio']>0.50) & (df2['hRatio']<0.75)] = 'moderate'
df2.loc[:,'sentiment'][(df2['hRatio']>0.75) & (df2['hRatio']<= 1)] = 'helpful'
train = df2[0:10000]

In [10]:
samp = train['reviewText'][0]
def to_words(review):
    letters = re.sub("[^a-zA-Z]", " ", review)
    words = letters.lower().split()
    good_words = [w for w in words if not w in stopwords.words("english")]
    return( " ".join( good_words ))   

clean_review = to_words( samp )
print clean_review

print "Cleaning and parsing the training set movie reviews...\n"
train_rvws = []
num_rvws = train['reviewText'].size
for i in xrange( 0, num_rvws ):
    if( (i+1)%1000 == 0 ):
        print "Review %d of %d\n" % ( i+1, num_rvws )                                                                    
    train_rvws.append( to_words( train["reviewText"][i] ))


liked quality lens built light lens discernable distortion anywhere magnified everything evenly without ripples distortion seen low cost magnifiers light nice touch easy use want pull lens bit focused close center look provides nice even coverage like brightness actually dimmness light focused leds lots brighter know seen also light focuses center field view lens close focused properly bottom line good value magnifier could made great better quality control btw feel honest effective reviews take place first hand experiences lacking online shopping always appreciated help received reviewers work hard return favor best hope found review helpful anything thought lacking unclear leave comment fix
Cleaning and parsing the training set movie reviews...

Review 1000 of 10000

Review 2000 of 10000

Review 3000 of 10000

Review 4000 of 10000

Review 5000 of 10000

Review 6000 of 10000

Review 7000 of 10000

Review 8000 of 10000

Review 9000 of 10000

Review 10000 of 10000



In [ ]:
#Make a bag of words through pure python
sentences = [
    "I like pie and chocolate",
    "Pie and oranges are fantastic and great.",
    "Wow pie and ketchup is a nasty combination!",
    "100 pies on a plate and she can't choose a single one.",
    "Who dun did it? Pie."
]

#find all the unique words in all sentences
#join all sentences, each separated by a space, then split all words into a list
set(" ".join(sentences).split(" "))

In [11]:
#Get the features to train the decision trees
len(train_rvws)
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 1000) 
train_data_features = vectorizer.fit_transform(train_rvws)
train_data_features = train_data_features.toarray()

In [12]:
#See what words you get.
print train_data_features.shape
vocab = vectorizer.get_feature_names()
print vocab
# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print count, tag

(10000L, 1000L)
[u'able', u'absolutely', u'absorption', u'accurate', u'acid', u'acids', u'acne', u'across', u'action', u'active', u'actually', u'add', u'added', u'addition', u'age', u'ago', u'aid', u'air', u'alcohol', u'allergic', u'allergies', u'allergy', u'almost', u'alone', u'along', u'already', u'also', u'alternative', u'although', u'always', u'amazing', u'amazon', u'amino', u'amount', u'another', u'anti', u'antibiotics', u'anxiety', u'anymore', u'anyone', u'anything', u'anyway', u'anywhere', u'applied', u'apply', u'area', u'areas', u'around', u'arrived', u'asleep', u'available', u'average', u'avoid', u'away', u'baby', u'back', u'bacteria', u'bad', u'bag', u'bar', u'bars', u'base', u'based', u'basically', u'bath', u'bathroom', u'batteries', u'battery', u'beard', u'become', u'bed', u'began', u'behind', u'believe', u'benefit', u'benefits', u'best', u'better', u'big', u'bit', u'black', u'blade', u'blades', u'blood', u'blue', u'body', u'bottle', u'bottles', u'bottom', u'bought', u'box'

In [13]:
print "Training the random forest..."
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, train["sentiment"] )

Training the random forest...


In [20]:
test = df2[25000:35000]
test.index = range(10000)
test_rvws = []
num_rvws = test['reviewText'].size
print num_rvws
for i in xrange(0, num_rvws):
    if( (i+1)%1000 == 0 ):
        print "Review %d of %d\n" % ( i+1, num_rvws )                                                                    
    test_rvws.append( to_words( test["reviewText"][i] ))

test_data_features = vectorizer.transform(test_rvws)
test_data_features = test_data_features.toarray()
result = forest.predict(test_data_features)
output = pd.DataFrame( data={"id":test["asin"], "sentiment":result} )

10000
Review 1000 of 10000

Review 2000 of 10000

Review 3000 of 10000

Review 4000 of 10000

Review 5000 of 10000

Review 6000 of 10000

Review 7000 of 10000

Review 8000 of 10000

Review 9000 of 10000

Review 10000 of 10000



In [44]:
print output.head()
print test[['asin','sentiment']].head()
count = 0.0
for i in xrange(0,len(output)):
    if test['sentiment'][i] == output['sentiment'][i]:
        count += 1
acc = 100*(count/len(output))
print "Model is {}% accurate".format(acc)

           id sentiment
0  B001OBZG6C   helpful
1  B001OBZHGG   helpful
2  B001OBZHGG   helpful
3  B001OCEVAS   helpful
4  B001OCEVAS   helpful
         asin sentiment
0  B001OBZG6C   helpful
1  B001OBZHGG   helpful
2  B001OBZHGG   helpful
3  B001OCEVAS  moderate
4  B001OCEVAS   helpful
Model is 69.53% accurate


https://www.analyticsvidhya.com/blog/2016/04/complete-tutorial-tree-based-modeling-scratch-in-python/#one

Learn about Decision Trees before implementing it